In [1]:
import pandas as pd
import sys
sys.path.append("..")
from src.Eval import eval_llm

import os
import io
from contextlib import redirect_stdout


In [2]:
import sys
sys.path.append("..")
from src.Eval import eval_llm

import os
import io
from contextlib import redirect_stdout

def clean_dict(d):
    """
    Recursively remove keys from a dictionary if they map to empty lists.
    """
    if isinstance(d, dict):
        return {k: clean_dict(v) for k, v in d.items() if clean_dict(v) != {} and v != []}
    elif isinstance(d, list):
        return [clean_dict(item) for item in d if clean_dict(item) != {} and item != []]
    else:
        return d

def parse_filename(filename):
    parts = filename.replace(".jsonl", "").split("_")
    dataset = parts[0]
    k_shot = parts[1].split(".")[0]
    k = int(k_shot.replace("shot", ""))
    if k and k != 0 and len(parts) == 3:
        sampling_method = parts[2]
    else:
        sampling_method = "rand"
    return dataset, k, sampling_method

def silent_eval_llm(fpath):
    with io.StringIO() as buf, redirect_stdout(buf):
        # Call the actual eval_llm function
        return eval_llm(fpath)
    
def aggregated_results_llm(
    results_dir,
    ent_types=["overall", "Artist", "WoA"],
    eval_schemas=["strict", "exact", "ent_type"],
    metrics=["f1", "f1_macro", "f1_micro", "precision", "precision_macro", "recall", "recall_macro", "missed", "spurious", "incorrect"],
    datasets=["dataset1", "dataset2", "dataset3", "dataset4", "dataset5"],
    sampling_methods=["rand", "tfidf", ""],
    ks=[0,5,15,25,35,45]
):
    results = {}
    for schema in eval_schemas:
        results[schema] = {}
        for ent_type in ent_types:
            results[schema][ent_type] = {}
            for metric in metrics:
                results[schema][ent_type][metric] = {}
                for model in [m for m in os.listdir(results_dir) if m != "archive"]:
                    results[schema][ent_type][metric][model] = {}
                    for sampling_method in sampling_methods:
                        results[schema][ent_type][metric][model][sampling_method] = {}
                        for k in ks:
                            results[schema][ent_type][metric][model][sampling_method][k] = []

    model_dirs = [
        os.path.join(results_dir, d)
        for d in os.listdir(results_dir)
        if os.path.isdir(os.path.join(results_dir, d)) and d != "archive"
    ]

    for model_dir in model_dirs:
        model = model_dir.split(os.sep)[-1]
        files = [
            os.path.join(model_dir, f)
            for f in os.listdir(model_dir)
            if os.path.isfile(os.path.join(model_dir, f))
        ]

        for fpath in files:
            print(fpath)
            filename = os.path.basename(fpath)
            dataset, k, sampling_method = parse_filename(filename)

            if dataset in datasets and k in ks:
                predictions = silent_eval_llm(fpath)

                for ent_type in ent_types:
                    for schema in eval_schemas:
                        for metric in metrics:
                            key = f"{ent_type}_{schema}_{metric}"
                            if key in predictions.keys():
                                results[schema][ent_type][metric][model][sampling_method][k].append(predictions[key])
    return clean_dict(results)

results = aggregated_results_llm("../output/reddit+shsyt/")
#results_tfidf = aggregated_results_llm("../output/tfidf_sampling")



2024-09-07 07:16:47 root INFO: Imported 658 predictions for 658 true examples


../output/reddit+shsyt/llama3.1/dataset4_0shot.jsonl


2024-09-07 07:16:47 root INFO: Imported 658 predictions for 658 true examples


../output/reddit+shsyt/llama3.1/dataset5_0shot.jsonl
../output/reddit+shsyt/llama3.1/dataset3_0shot.jsonl


2024-09-07 07:16:47 root INFO: Imported 658 predictions for 658 true examples
2024-09-07 07:16:47 root INFO: Imported 660 predictions for 660 true examples


...

../output/reddit+shsyt/firefunction-v2/dataset4_5shot_rand.jsonl


2024-09-07 07:16:57 root INFO: Imported 658 predictions for 658 true examples


../output/reddit+shsyt/firefunction-v2/dataset5_25shot_tfidf.jsonl


2024-09-07 07:16:57 root INFO: Imported 660 predictions for 660 true examples


../output/reddit+shsyt/firefunction-v2/dataset2_25shot_tfidf.jsonl


In [3]:
import pandas as pd
import numpy as np

def results_to_dataframe(aggregated_results, agg_func='mean'):
    data = []

    for schema, schema_dict in aggregated_results.items():
        for ent_type, ent_type_dict in schema_dict.items():
            for metric, metric_dict in ent_type_dict.items():
                for model, model_dict in metric_dict.items():
                    for sampling, sampling_dict in model_dict.items():
                        for k_shot, values in sampling_dict.items():
                            if values:
                                if agg_func == 'mean':
                                    agg_value = np.mean(values)
                                elif agg_func == 'sum':
                                    agg_value = np.sum(values)
                                else:
                                    raise ValueError("Invalid aggregation function. Use 'mean' or 'sum'.")
                                
                                data.append([schema, ent_type, metric, model, sampling, k_shot, agg_value])

    df = pd.DataFrame(data, columns=['Schema', 'Entity Type', 'Metric', 'Model', 'Sampling', 'k', 'Value'])
    df.Model = df.Model.str.replace("mistral", "Mistral-7B").str.replace("mixtral", "Mixtral-8x22B").str.replace("gpt-3.5-turbo-0125", "GPT-3.5-Turbo")
    return df.set_index(['Schema', 'Entity Type', 'Metric', 'Model', 'Sampling', 'k']).unstack(['Schema', 'Entity Type', 'Metric'])

results_to_dataframe(results)


Value                                         \
Schema                         strict                                          
Entity Type                   overall                                          
Metric                       f1_macro  f1_micro precision_macro recall_macro   
Model           Sampling k                                                     
firefunction-v2 rand     5   0.752028  0.754567        0.761079     0.743824   
                         25  0.764807  0.765908        0.778371     0.752175   
                tfidf    5   0.758773  0.761246        0.768487     0.749964   
                         25  0.776149  0.778070        0.781329     0.771571   
llama3.1        rand     0   0.730728  0.733185        0.727095     0.734732   
                         5   0.739356  0.742094        0.725537     0.757064   
                         25  0.587314  0.588008        0.702230     0.504768   
llama3.1:70b    rand     0   0.726988  0.728105        0.681858     0.783480   
                         5   0.738566  0.739220        0.693800     0.794619   
                         25  0.761770  0.762839        0.726488     0.803251   
                tfidf    5   0.751967  0.753064        0.708747     0.804094   
                         25  0.768406  0.769336        0.728974     0.815500   

                                                                           \
Schema                                                                      
Entity Type                    Artist                                       
Metric                             f1 precision    recall missed spurious   
Model           Sampling k                                                  
firefunction-v2 rand     5   0.777905  0.775880  0.780395   56.6    59.20   
                         25  0.777896  0.786833  0.769788   67.8    60.20   
                tfidf    5   0.782906  0.782945  0.783378   53.0    53.80   
                         25  0.796028  0.791832  0.800731   53.6    58.00   
llama3.1        rand     0   0.760430  0.761950  0.759039   67.5    66.25   
                         5   0.803235  0.820937  0.786280   62.0    46.00   
                         25  0.659004  0.781818  0.569536   55.0    14.00   
llama3.1:70b    rand     0   0.768410  0.750805  0.787439   43.0    60.75   
                         5   0.777437  0.763449  0.792534   43.4    57.40   
                         25  0.787525  0.775762  0.800123   47.0    58.40   
                tfidf    5   0.782619  0.764431  0.802134   40.8    58.60   
                         25  0.796981  0.783633  0.810967   40.5    52.50   

                                       ...                            \
Schema                                 ...  ent_type                   
Entity Type                            ...    Artist                   
Metric                      incorrect  ...    recall missed spurious   
Model           Sampling k             ...                             
firefunction-v2 rand     5      20.40  ...  0.801034   56.6    59.20   
                         25     13.60  ...  0.790731   67.8    60.20   
                tfidf    5      23.00  ...  0.805271   53.0    53.80   
                         25     16.40  ...  0.824112   53.6    58.00   
llama3.1        rand     0      15.75  ...  0.786751   67.5    66.25   
                         5      19.00  ...  0.796834   62.0    46.00   
                         25     10.00  ...  0.629139   55.0    14.00   
llama3.1:70b    rand     0      30.25  ...  0.810602   43.0    60.75   
                         5      29.40  ...  0.818236   43.4    57.40   
                         25     23.00  ...  0.829204   47.0    58.40   
                tfidf    5      28.60  ...  0.823325   40.8    58.60   
                         25     25.00  ...  0.837026   40.5    52.50   

                                                                            \
Schema                                                    

In [4]:
results

{'strict': {'overall': {'f1_macro': {'llama3.1': {'rand': {0: [0.7541778460519967,
       0.7161413110210668,
       0.7239345509893454,
       0.7286579278457901],
      5: [0.7393558703508001],
      25: [0.5873144157088122]}},
    'llama3.1:70b': {'rand': {0: [0.7456525363678046,
       0.7208214980394578,
       0.7166172106824926,
       0.7248613037447988],
      5: [0.7359636283232738],
      25: [0.752461467678859]},
     'tfidf': {5: [0.7583700111898546], 25: [0.7521098838855849]}},
    'firefunction-v2': {'rand': {5: [0.7409455944891575,
       0.7604224308300396,
       0.7435115967030861,
       0.7593963739007707,
       0.7558636253727734],
      25: [0.7545338600032194,
       0.7957792964199737,
       0.7629514363119361,
       0.769948175198443,
       0.7408210269611164]},
     'tfidf': {5: [0.7517304098666999,
       0.74114878915219,
       0.7580474142242672,
       0.753341756709655,
       0.7895957909450959],
      25: [0.7692518874399451,
       0.799188592279

# Precision

In [5]:
_results = results_to_dataframe(results, "precision", "mean").reset_index()
_results[(_results.Schema == "strict") & (_results.k == 0)].drop(["Schema", "k"], axis=1)


TypeError: results_to_dataframe() takes from 1 to 2 positional arguments but 3 were given

# Recall

In [ ]:
_results = results_to_dataframe(results, "recall", "mean").reset_index()
_results[(_results.Schema == "strict") & (_results.k == 0)].drop(["Schema", "k"], axis=1)


,Entity Type,Model,Value
0,Artist,llama3.1,0.756232
5,Artist,llama3.1-70b,0.779171
6,WoA,llama3.1,0.716194
11,WoA,llama3.1-70b,0.775184


# F1

In [ ]:
results_to_dataframe(results, "f1_macro", "mean")


Value
Schema   Entity Type Model        k           
strict   overall     llama3.1     0   0.736789
                                  5   0.736789
                                  15  0.736789
                                  25  0.736789
                                  35  0.736789
                     llama3.1-70b 0   0.725325
exact    overall     llama3.1     0   0.751727
                                  5   0.751727
                                  15  0.751727
                                  25  0.751727
                                  35  0.751727
                     llama3.1-70b 0   0.758958
ent_type overall     llama3.1     0   0.779673
                                  5   0.779673
                                  15  0.779673
                                  25  0.779673
                                  35  0.779673
                     llama3.1-70b 0   0.768855

In [ ]:
results_to_dataframe(results, "f1", "mean")


Value
Schema   Entity Type Model        k          
strict   Artist      llama3.1-70b 0  0.771081
                     llama3.1-8b  0  0.763753
         WoA         llama3.1-70b 0  0.679570
                     llama3.1-8b  0  0.709825
exact    Artist      llama3.1-70b 0  0.833947
                     llama3.1-8b  0  0.784851
         WoA         llama3.1-70b 0  0.683969
                     llama3.1-8b  0  0.718603
ent_type Artist      llama3.1-70b 0  0.793239
                     llama3.1-8b  0  0.788424
         WoA         llama3.1-70b 0  0.744471
                     llama3.1-8b  0  0.770921